# Impact Data

In [1]:
import pandas as pd
import numpy as np

In [ ]:
c = pd.read_csv("/home/syed/Documents/Master_Thesis/bitflow4j/target/out-8.csv")
c.tail(5)

In [ ]:
c.head(5)

- QoS ~ number of frames/sec = bytes/s / no of open streams
- Qos Estimate of whole system = QoS / no of client nodes
- Hint: Aggregate last 10 values of client.bin file before applying formula. Drop in value means change in system.

In [ ]:
c1 = c.tail(10)
bytes_s = c1['bytes/s'].median()
streams = c1['streams'][c1.index[-1]]
qos = bytes_s/streams
qos

In [ ]:
c1 = c.tail()
QoS = c['bytes/s']/c['streams']

In [ ]:
# Remove data when stream equals zero.
c = c[c['streams'] != 0]
streams = np.array(list(c.groupby(['streams'])['bytes/s'].groups.keys()))
print(streams)

In [ ]:
mean = c.groupby(['streams'])['bytes/s'].mean()
mean

In [ ]:
std = c.groupby(['streams'])['bytes/s'].std()
std

# Visualizing Log files

In [3]:
# Anamoly: Stress CPU
# Action: Soft Reboot
# Time in seconds
# action_time: Action time from trigger from EC to SS and changes in OS
# service_restart_time: restart docker services: nginx server and injector using ansibl
# get_ip_time: soft reboot is called with server name, this is where we get public/private ips from server name 
# total time: total time of this process
# QoS1: calculated Quality of Service before running soft reboot
# QoS2: calculated quality of service after running soft reboot
# Impact: QoS - Qos2= A smaller Qos value means a positive impact

df_softreboot = pd.read_csv("log_files/softreboot_cpustress.csv")
df_softreboot.tail(1)

,Action,action_time,service_restart_time,get_ip_time,total_time,status,QoS1,Qos2,Impact,tag
12,soft_reboot,17,28,27.49784,75.13969,success,99764.171074,68100.492674,31663.6784,CPU_stress


In [4]:
# Aggregating log files for soft reboot
softreboot_aggregate = df_softreboot.mean()
softreboot_aggregate

action_time                 19.615385
service_restart_time        28.615385
get_ip_time                 34.678530
total_time                  86.335632
QoS1                    120783.533709
Qos2                     64984.344064
Impact                   55799.189645
dtype: float64

In [5]:
df_hardreboot = pd.read_csv("log_files/hardreboot_cpustress.csv")
df_hardreboot.tail(1)

,action,action_time,service_restart_time,get_ip_time,total_time,status,QoS1,QoS2,Impact,tag
4,hard_reboot,17,28,27.970046,75.884004,success,104016.83954,105421.394957,-1404.555416,CPU_Stress


In [6]:
hardreboot_aggregate = df_hardreboot.mean()
hardreboot_aggregate

action_time                 16.800000
service_restart_time        31.600000
get_ip_time                 27.587051
total_time                  79.066321
QoS1                    101612.803272
QoS2                     78667.392984
Impact                   22945.410288
dtype: float64

In [7]:
df_resize = pd.read_csv("log_files/resize_cpustress.csv")
df_resize.tail(1)

,action,action_time,resize_confirm_time,total_time,ansible_playbook_run,QoS1,QoS2,impact,tag
8,Server_resize,25,6,103.9659,32s,231697.419424,245128.144692,13430.725268,CPU_Stress


In [8]:
resize_aggregate = df_resize.mean()
resize_aggregate

action_time                23.333333
resize_confirm_time         5.888889
total_time                105.695841
QoS1                   232827.374853
QoS2                   245529.123522
impact                  15833.162217
dtype: float64

In [9]:
df_restartservice = pd.read_csv("log_files/restartservice_cpustress.csv")
df_restartservice.tail(1)

,action,total_time,action_time,QoS1,QoS2,Impact,tag
4,Restart_service,7.152557,7.152557,244244.131379,228431.607639,15812.523741,CPU_Stress


In [10]:
restartservice_aggregate = df_restartservice.mean()
restartservice_aggregate

total_time          4.291534
action_time         4.291534
QoS1           240280.197620
QoS2           235981.216708
Impact           4298.980912
dtype: float64

In [11]:
df_scaleUp = pd.read_csv("log_files/scaleup_cpustress.csv")
df_scaleUp.tail(1)

,action,total_time,action_create_new_vm,action_assign_floating_ip,action_ansible_run,get_ip,QoS1,QoS2,impact,tag
4,Scale Up,177.007808,32,7,67,34.600375,269531.51612,269598.729591,-67.213471,CPU_Stress


In [12]:
scaleUp_aggregate = df_scaleUp.mean()
scaleUp_aggregate

total_time                      181.262550
action_create_new_vm             29.200000
action_assign_floating_ip         8.800000
action_ansible_run               58.600000
get_ip                           34.034492
QoS1                         223337.836279
QoS2                         211484.634594
impact                        11853.201685
dtype: float64

In [13]:
df_migrate = pd.read_csv("log_files/migrate_cpustress.csv")
df_migrate.tail(1)

,action,total_time,action_create_vm_snapshot,action_create_new_vm,assign_floating_ip,action_run_ansible,QoS1,QoS2,Impact,tag
4,Migrate,311.925287,125,40,11,11,200310.360319,264678.901869,-64368.541551,CPU_Stress


In [14]:
migrate_aggregate = df_migrate.mean()
migrate_aggregate

total_time                      311.925287
action_create_vm_snapshot       125.000000
action_create_new_vm             40.000000
assign_floating_ip               11.000000
action_run_ansible               11.000000
QoS1                         200310.360319
QoS2                         264678.901869
Impact                       -64368.541551
dtype: float64

### Adding impact percentage to each dataframe

In [15]:
(1 - 88186.996485/107019.450679) * 100

17.597225620683755

In [16]:
# calculate impact as % for each dataframe add a new column
# >1 : positive impact
# 0 means no impact
# <1 : negative impact

impact_percentage_migrate = []
for index, row in df_migrate.iterrows():
    val = (1 - row['QoS2']/row['QoS1'] ) * 100
    impact_percentage_migrate.append(val)
    
df_migrate['impact_percentage'] = impact_percentage_migrate

impact_percentage_scaleUp = []
for index, row in df_scaleUp.iterrows():
    val = (1 - row['QoS2']/row['QoS1'] ) * 100
    impact_percentage_scaleUp.append(val)
    
df_scaleUp['impact_percentage'] = impact_percentage_scaleUp

impact_percentage_softreboot = []
for index, row in df_softreboot.iterrows():
    if row['Qos2']:
        val =(1 - row['Qos2']/row['QoS1'] ) * 100
    else: 
        pass
    impact_percentage_softreboot.append(val)
    
df_softreboot['impact_percentage'] = impact_percentage_softreboot

impact_percentage_hardreboot = []
for index, row in df_hardreboot.iterrows():
    if row['QoS2']:
        val =(1 - row['QoS2']/row['QoS1'] ) * 100
    else: 
        pass
    impact_percentage_hardreboot.append(val)
    
df_hardreboot['impact_percentage'] = impact_percentage_hardreboot

impact_percentage_resize = []
for index, row in df_resize.iterrows():
    if row['QoS2']:
        val = ((1 - row['QoS2']/row['QoS1'] ) * 100) * -1
    else: 
        pass
    impact_percentage_resize.append(val)
    
df_resize['impact_percentage'] = impact_percentage_resize

impact_percentage_restartservice = []
for index, row in df_restartservice.iterrows():
    if row['QoS2']:
        val = ((1 - row['QoS2']/row['QoS1'] ) * 100) * -1
    else: 
        pass
    impact_percentage_restartservice.append(val)
    
df_restartservice['impact_percentage'] = impact_percentage_restartservice

df_softreboot = df_softreboot[df_softreboot['impact_percentage'] != 0]

print('sr:  ' + str(df_softreboot['impact_percentage'].median()))
print('hr:  ' + str(df_hardreboot['impact_percentage'].median()))
print('res:  ' + str(df_resize['impact_percentage'].median()))
print('srest:  ' + str(df_restartservice['impact_percentage'].median()))
print('mig:  ' + str(df_migrate['impact_percentage'].median()))
print('sup:  ' + str(df_scaleUp['impact_percentage'].median()))

sr:  8.793279976925184
hr:  13.284447096202967
res:  5.796665885070307
srest:  -0.865045016937871
mig:  -32.13440455518462
sup:  -0.40512891591497624


# Weighted Sum Model

In [17]:
#First look at weighted sum model

#### Weighted sum model

In [18]:
dataframe = [df_softreboot,df_hardreboot,df_resize,df_migrate,df_scaleUp,df_restartservice]
df_index = ["soft_reboot","hard_reboot","resize","migrate","scaleUp","restart_service"]

# Mean time to recover (MTTR) =total_time
MTTR = []
for df in dataframe:
    MTTR.append(df['total_time'].mean())
print(MTTR)

Impact = []
for df in dataframe:
    Impact.append(df['impact_percentage'].median())
print(Impact)


#list_for_df = [df_index,MTTR,Impact]
#df = pd.DataFrame(list_for_df)
df = pd.DataFrame()
df[0] = df_index
df[1] = MTTR
df[2] = Impact
df.columns = ['Alternatives', 'MTTR','Impact']
model = df
df.set_index('Alternatives',inplace=True)
df

#df = df.pivot_table('Value', ['LOCATION'], 'TIME')

    

[82.03684260845185, 79.06632132530213, 105.69584070311652, 311.92528724670416, 181.26254997253417, 4.291534423828125]
[8.793279976925184, 13.284447096202967, 5.796665885070307, -32.13440455518462, -0.40512891591497624, -0.865045016937871]


,MTTR,Impact
Alternatives,,
soft_reboot,82.036843,8.793280
hard_reboot,79.066321,13.284447
resize,105.695841,5.796666
migrate,311.925287,-32.134405
scaleUp,181.262550,-0.405129
restart_service,4.291534,-0.865045


In [19]:
# since the formula dictates a positive value for calculation we multiply all values of MTTR to -1 to get the correct result
model['MTTR'] = model['MTTR'].apply(lambda x: x*-1)

# Adding weight
MTTR_weight= 0.6
Impact_weight= 0.4
#model = model.reset_index()


# Calculating Weighted sum model scores for each alternative
A1 = (model['MTTR'][0] * MTTR_weight) + (model['Impact'][0] * Impact_weight) 
A2 = (model['MTTR'][1] * MTTR_weight) + (model['Impact'][1] * Impact_weight)
A3 = (model['MTTR'][2] * MTTR_weight) + (model['Impact'][2] * Impact_weight)
A4 = (model['MTTR'][3] * MTTR_weight) + (model['Impact'][3] * Impact_weight)
A5 = (model['MTTR'][4] * MTTR_weight) + (model['Impact'][4] * Impact_weight)
A6 = (model['MTTR'][5] * MTTR_weight) + (model['Impact'][5] * Impact_weight)

print(A1,A2,A3,A4,A5,A6)

-45.704793574301036 -42.12601395670009 -61.09883806784178 -200.00893417009632 -108.91958154988649 -2.9209386610720234


### Problem here is that we are comparing apples with oranges

In [20]:
#Proposed solution: Separate weigted sum models for MTTR, Impact, Success probability,number of actions, main action time
# 1, Based on MTTR: Taking two nginx servers here

#Normalization 
# Max and Min values for MTTR
# Max and Min for impact
# Normalize and return each to range 0-`99
# Save the volations for future: important. Thresholds
#MTTR_min= model['MTTR'].min()
MTTR_min = 0
MTTR_max= model['MTTR'].max()
#diff_MTTR = (MTTR_max-MTTR_min)/100
scaled_MTTR = []
for val in MTTR:
    # since you know we need high vals
    scaled_val = (1 - (val/MTTR_max)) * 100
    scaled_MTTR.append(scaled_val)

Impact_min= model['Impact'].min()
Impact_max= model['Impact'].max()

OldRange = (Impact_max - Impact_min)  
NewRange = (100 - 0)  

scaled_Impact = []
for res in Impact:
    NewValue = (((res - Impact_min) * NewRange) / OldRange) + 0
    scaled_Impact.append(NewValue)
    
#print(scaled_Impact)
#print(MTTR)

#now happily apply the NWSM
n_model = pd.DataFrame()
n_model[0] = df_index
n_model[1] = scaled_MTTR
n_model[2] = scaled_Impact
n_model.columns = ['Alternatives', 'MTTR','Impact']
n_model.set_index('Alternatives',inplace=True)
n_model



,MTTR,Impact
Alternatives,,
soft_reboot,2011.596984,90.111667
hard_reboot,1942.378821,100.000000
resize,2562.891597,83.513935
migrate,7368.386000,0.000000
scaleUp,4323.723547,69.859264
restart_service,200.000000,68.846654


In [21]:
MTTR_weight= 0.6
Impact_weight= 0.4

n_model['MTTR'] = n_model['MTTR']*0.6
n_model['Impact'] = n_model['MTTR']*0.4

n_model

,MTTR,Impact
Alternatives,,
soft_reboot,1206.958190,482.783276
hard_reboot,1165.427293,466.170917
resize,1537.734958,615.093983
migrate,4421.031600,1768.412640
scaleUp,2594.234128,1037.693651
restart_service,120.000000,48.000000


In [22]:
# Adding weight
MTTR_weight= 0.6
Impact_weight= 0.4
#model = model.reset_index()


# Calculating Weighted sum model scores for each alternative
A1 = (n_model['MTTR'][0] * MTTR_weight) + (n_model['Impact'][0] * Impact_weight) 
A2 = (n_model['MTTR'][1] * MTTR_weight) + (n_model['Impact'][1] * Impact_weight)
A3 = (n_model['MTTR'][2] * MTTR_weight) + (n_model['Impact'][2] * Impact_weight)
A4 = (n_model['MTTR'][3] * MTTR_weight) + (n_model['Impact'][3] * Impact_weight)
A5 = (n_model['MTTR'][4] * MTTR_weight) + (n_model['Impact'][4] * Impact_weight)
A6 = (n_model['MTTR'][5] * MTTR_weight) + (n_model['Impact'][5] * Impact_weight)

scores = [A1,A2,A3,A4,A5,A6]
n_model['Awsm-Score'] = scores
#print(A1,A2,A3,A4,A5,A6)
#print("Winner is Hard Reboot with alternative sum:" + str(A2))
n_model


,MTTR,Impact,Awsm-Score
Alternatives,,,
soft_reboot,1206.958190,482.783276,917.288225
hard_reboot,1165.427293,466.170917,885.724742
resize,1537.734958,615.093983,1168.678568
migrate,4421.031600,1768.412640,3359.984016
scaleUp,2594.234128,1037.693651,1971.617937
restart_service,120.000000,48.000000,91.200000


In [23]:
# Adding sucess probability,action time, no of steps
# Sucess probability not calculated here: but can explain the method

#df_index = ["soft_reboot","hard_reboot","resize","migrate","scaleUp","restart_service"]
sucess_probability=[95,100,85,90,0,0]
no_of_steps=[3,3,4,6,5,1]
main_action_time=[]

main_action_time.append(df_softreboot['action_time'].mean())
main_action_time.append(df_hardreboot['action_time'].mean())
main_action_time.append(df_resize['action_time'].mean()) 
main_action_time.append(df_migrate['action_create_vm_snapshot'].mean()) 
main_action_time.append(df_scaleUp['action_create_new_vm'].mean()) 
main_action_time.append(df_restartservice['total_time'].mean()) #same as action time

#Normalize
scaled_noofsteps = []
for lol in no_of_steps:
    NewValue = (((lol - min(no_of_steps)) * 100) / 5) + 0 # formula in last block
    scaled_noofsteps.append(NewValue)

scaled_main_action_time = []
for rz in main_action_time:
    NewValue = (((rz - min(main_action_time)) * 100) / 120.70846557617188) + 0 # formula in last block
    scaled_main_action_time.append(NewValue)
    
#create new NWSM
add_model = pd.DataFrame()
add_model[0] = df_index
add_model[1] = scaled_MTTR
add_model[2] = scaled_Impact
add_model[3] = sucess_probability
add_model[4] = scaled_noofsteps
add_model[5] = scaled_main_action_time
add_model.columns = ['Alternatives', 'MTTR','Impact','sucess_probability','scaled_noofsteps','scaled_main_action_time']

print(add_model)

#new weights
sucess_probability_weight = 0.5
MTTR_weight= 0.2
Impact_weight= 0.1
scaled_noofsteps_weight=0.1
scaled_main_action_time_weight=0.1

#calculating with 5 criterias 
A1 = (add_model['MTTR'][0] * MTTR_weight) + (add_model['Impact'][0] * Impact_weight) + (add_model['sucess_probability'][0] * sucess_probability_weight) + (add_model['scaled_main_action_time'][0] * scaled_noofsteps_weight) + (add_model['scaled_noofsteps'][0] * scaled_main_action_time_weight) 
A3 = (add_model['MTTR'][2] * MTTR_weight) + (add_model['Impact'][2] * Impact_weight) + (add_model['sucess_probability'][2] * sucess_probability_weight) + (add_model['scaled_main_action_time'][2] * scaled_noofsteps_weight) + (add_model['scaled_noofsteps'][2] * scaled_main_action_time_weight) 
A4 = (add_model['MTTR'][3] * MTTR_weight) + (add_model['Impact'][3] * Impact_weight) + (add_model['sucess_probability'][3] * sucess_probability_weight) + (add_model['scaled_main_action_time'][3] * scaled_noofsteps_weight) + (add_model['scaled_noofsteps'][3] * scaled_main_action_time_weight) 
A5 = (add_model['MTTR'][4] * MTTR_weight) + (add_model['Impact'][4] * Impact_weight) + (add_model['sucess_probability'][4] * sucess_probability_weight) + (add_model['scaled_main_action_time'][4] * scaled_noofsteps_weight) + (add_model['scaled_noofsteps'][4] * scaled_main_action_time_weight) 
A6 = (add_model['MTTR'][5] * MTTR_weight) + (add_model['Impact'][5] * Impact_weight) + (add_model['sucess_probability'][5] * sucess_probability_weight) + (add_model['scaled_main_action_time'][5] * scaled_noofsteps_weight) + (add_model['scaled_noofsteps'][5] * scaled_main_action_time_weight) 

scores = [A1,A2,A3,A4,A5,A6]

#print(A1,A2,A3,A4,A5,A6)
#print("Winner is Hard Reboot with alternative sum:" + str(A2))
#max(main_action_time)-min(main_action_time)

add_model['Awsm-score'] = scores
add_model.set_index('Alternatives',inplace=True)
add_model

      Alternatives         MTTR      Impact  sucess_probability  \
0      soft_reboot  2011.596984   90.111667                  95   
1      hard_reboot  1942.378821  100.000000                 100   
2           resize  2562.891597   83.513935                  85   
3          migrate  7368.386000    0.000000                  90   
4          scaleUp  4323.723547   69.859264                   0   
5  restart_service   200.000000   68.846654                   0   

   scaled_noofsteps  scaled_main_action_time  
0              40.0                11.025296  
1              40.0                10.362542  
2              60.0                15.775032  
3             100.0               100.000000  
4              80.0                20.635227  
5               0.0                 0.000000  


,MTTR,Impact,sucess_probability,scaled_noofsteps,scaled_main_action_time,Awsm-score
Alternatives,,,,,,
soft_reboot,2011.596984,90.111667,95,40.0,11.025296,463.933093
hard_reboot,1942.378821,100.000000,100,40.0,10.362542,885.724742
resize,2562.891597,83.513935,85,60.0,15.775032,571.007216
migrate,7368.386000,0.000000,90,100.0,100.000000,1538.677200
scaleUp,4323.723547,69.859264,0,80.0,20.635227,881.794158
restart_service,200.000000,68.846654,0,0.0,0.000000,46.884665


In [24]:
# how a new value changes things - how to make this continious / seamless

#### Lets try some other stuff

In [284]:
import math
# Technique for Order of Preference by Similarity to Ideal Solution (TOPSIS) is a multi-criteria decision analysis 
# method
# TOPSIS is based on the concept that the chosen alternative should have the shortest geometric
# distance from the positive ideal solution (PIS)[4] and the longest geometric distance from the negative ideal 
# solution (NIS)

#### ---- TOPSIS Application -----------####

# Step 1: Create an evaluation matrix consisting of m alternatives and n criteria, with the intersection of each alternative and criteria given as {\displaystyle x_{ij}} x_{ij}, we therefore have a matrix {\displaystyle (x_{ij})_{m\times n}} (x_{{ij}})_{{m\times n}}

topsis_model = pd.DataFrame()
topsis_model[0] = df_index
topsis_model[1] = MTTR
topsis_model[2] = Impact
topsis_model[3] = sucess_probability
topsis_model[4] = no_of_steps
topsis_model[5] = main_action_time
topsis_model.columns = ['Alternatives', 'MTTR','Impact','sucess_probability','no_of_steps','main_action_time']

#topsis_model

# Step 2: Normalization function to normalize matrix x to r
for index, row in df_migrate.iterrows():
    val = (1 - row['QoS2']/row['QoS1'] ) * 100
    impact_percentage_migrate.append(val)

rij = xij / math.sqrt(sum(m)*(x^2kj))

# Step 4: Add weights
# Step 5: Determine the worst and best alternative
# Step 6: Distance between target alternative and best and worst conditions
# Step 7: Calculate similarty
# Step 8: Rank the alternatives


,Alternatives,MTTR,Impact,sucess_probability,no_of_steps,main_action_time
0,soft_reboot,82.036843,8.793280,95,3,17.600000
1,hard_reboot,79.066321,13.284447,100,3,16.800000
2,resize,105.695841,5.796666,85,4,23.333333
3,migrate,311.925287,-32.134405,90,6,125.000000
4,scaleUp,181.262550,-0.405129,0,5,29.200000
5,restart_service,4.172325,-94.329593,0,1,4.291534
